In [1]:
import re
import string
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import multiprocessing
import json
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForTokenClassification
import concurrent.futures
import os
import gzip


# Load the ONNX model for NER Classification
tokenizer = AutoTokenizer.from_pretrained("/home/dhruv/NER/git/models/onnx_muril_model")
model = ORTModelForTokenClassification.from_pretrained('/home/dhruv/NER/git/models/onnx_muril_model')


ner_pipeline = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy='max')

/home/dhruv/grammer_checker/venv_grammer_check/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Reading the CSV file

In [2]:
file_name= 'combining_hindi_error_wiki'
#combining_hindi_error_wiki

In [3]:
# df = pd.read_csv(f"trigram_data/{file_name}.csv")
df1 = pd.read_csv(f"../{file_name}.csv")
df1 = df1[['Correct']]
df1.shape
# df = df[0:15001]
df1.head(2)

,Correct
0,महादेव बानी के रूप में और बाद में रियलि...
1,यह जर्मन दार्शनिक मार्टिन हाइडेगर के अस्त...


In [4]:
df1.columns

Index(['Correct'], dtype='object')

In [5]:
df = df1[500001:800001]

In [6]:
df

,Correct
500001,डीसीडब्ल्यू ने सात महाविद्यालयों को नोटिस जारी...
500002,"डीसीपी अधिकारी, रंजन चित्तौड़ा अपने गुरु के हत..."
500003,"डीसीपी को ये सारी बात पता होती है, पर वो इसके ..."
500004,डीसीबी 1988 में आठ जर्मन क्रिकेट क्लबों द्वारा...
500005,डीसूजा ने 1953 में पहले अंतर्राष्ट्रीय महिला ह...
...,...
799996,यही भावों का विरेचन है और कविता का यही वास्तवि...
799997,यही भाषा का सामान्य ज्ञान कहलाता है.
799998,यही भेद आज के सौलिसिटर तथा ऐडवोकेट में है.
799999,यही भौम सूर्य के परिक्रमा करते ग्रहों में मंगल...


### Preprocessing the datframe

In [7]:
import re
import pandas as pd

def preprocess_hindi(data, column_name):
    # New column for processed text
    new_column = 'pre_processed_sentence'
    
    # Create a copy of the original column to avoid modifying the original data
    data[new_column] = data[column_name]

    # Remove URLs (both http(s) and www)
    data[new_column] = data[new_column].str.replace(r'(https?://\S+|www\.\S+)', ' ', regex=True)
    print("Removed URLs")
    
    # Remove HTML tags
    data[new_column] = data[new_column].str.replace(r'<.*?>', ' ', regex=True)
    print("Removed HTML Tags")
    
    # Remove newline, carriage return, and tab characters by replacing them with space
    data[new_column] = data[new_column].str.replace(r'[\n\r\t]', ' ', regex=True)
    print("Removed tabs and new lines")
    
    # Define Hindi characters and punctuation pattern to keep
    hindi_pattern = r'[\u0900-\u097F\u0020\u0964\u0965\u0966-\u096F]'
    
    # Retain only Hindi characters and the specified punctuation
    data[new_column] = data[new_column].str.findall(hindi_pattern).str.join('')
    print("Kept only Hindi characters and specified punctuation")
    
    # Remove extra spaces and trim leading/trailing whitespace
    data[new_column] = data[new_column].str.replace(r'\s+', ' ', regex=True).str.strip()
    print("Removed extra spaces")
    
    # Drop rows with missing values in the new column and duplicates
    data.dropna(subset=[new_column], inplace=True)
    data.drop_duplicates(subset=[new_column], inplace=True)
    
    # Reset index
    data.reset_index(inplace=True, drop=True)
    
    return data


In [8]:
preprocess_hindi(df, 'Correct')#'correct_sentence')

/tmp/ipykernel_553358/943858414.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[new_column] = data[column_name].copy()
/tmp/ipykernel_553358/943858414.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[new_column] = data[new_column].apply(lambda x: re.sub(r'(https|http)?:\/\/\S+|www\.\S+', ' ', str(x)))
/tmp/ipykernel_553358/943858414.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Removed URLs
Removed HTML Tags


/tmp/ipykernel_553358/943858414.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[new_column] = data[new_column].apply(lambda x: re.sub(r'[\n\r\t]', ' ', str(x)))


Removed tabs and new lines


/tmp/ipykernel_553358/943858414.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[new_column] = data[new_column].apply(lambda x: ''.join(re.findall(hindi_pattern, str(x))))


Kept only Hindi characters and specified punctuation
Removed extra spaces


/tmp/ipykernel_553358/943858414.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[new_column] = data[new_column].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())
/tmp/ipykernel_553358/943858414.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=[new_column], inplace=True)
/tmp/ipykernel_553358/943858414.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

,Correct,pre_process_correct_sentence
0,डीसीडब्ल्यू ने सात महाविद्यालयों को नोटिस जारी...,डीसीडब्ल्यू ने सात महाविद्यालयों को नोटिस जारी...
1,"डीसीपी अधिकारी, रंजन चित्तौड़ा अपने गुरु के हत...",डीसीपी अधिकारी रंजन चित्तौड़ा अपने गुरु के हत्...
2,"डीसीपी को ये सारी बात पता होती है, पर वो इसके ...",डीसीपी को ये सारी बात पता होती है पर वो इसके क...
3,डीसीबी 1988 में आठ जर्मन क्रिकेट क्लबों द्वारा...,डीसीबी में आठ जर्मन क्रिकेट क्लबों द्वारा हस्त...
4,डीसूजा ने 1953 में पहले अंतर्राष्ट्रीय महिला ह...,डीसूजा ने में पहले अंतर्राष्ट्रीय महिला हॉकी ट...
...,...,...
299145,यही भावों का विरेचन है और कविता का यही वास्तवि...,यही भावों का विरेचन है और कविता का यही वास्तवि...
299146,यही भाषा का सामान्य ज्ञान कहलाता है.,यही भाषा का सामान्य ज्ञान कहलाता है
299147,यही भेद आज के सौलिसिटर तथा ऐडवोकेट में है.,यही भेद आज के सौलिसिटर तथा ऐडवोकेट में है
299148,यही भौम सूर्य के परिक्रमा करते ग्रहों में मंगल...,यही भौम सूर्य के परिक्रमा करते ग्रहों में मंगल...


In [9]:
# Droping the rows where the length is less than 3
df = df[df['pre_process_correct_sentence'].apply(lambda x: len(x.split()) >= 3)]

### Masking the sentence

In [10]:
# Define the function to get the predictions
def get_predictions(sentence):
    """
    Process a single sentence and replace named entities with 'MASK'.
    
    :param sentence: A string containing the sentence to process
    :return: A string with named entities replaced by 'MASK'
    """
    # Process the sentence using the NER pipeline
    entities = ner_pipeline(sentence)
    
    # Create masked sentence
    masked_sentence = sentence
    for item in sorted(entities, key=lambda x: x['start'], reverse=True):
        start, end = item['start'], item['end']
        masked_sentence = masked_sentence[:start] + "MASK" + masked_sentence[end:]
    
    return masked_sentence


In [11]:
def process_row(row):
    return get_predictions(row['pre_process_correct_sentence'])

In [12]:
def main():
    # Get the number of CPUs and set the number of workers to use
    num_cpus = os.cpu_count()
    num_workers = int(num_cpus * 0.8)
    
    # Apply the function to the DataFrame using multiprocessing
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Create a progress bar
        tqdm.pandas(desc="Processing")
        
        # Apply the function to each row in parallel
        df['Masked_sentence'] = list(tqdm(executor.map(process_row, [row for _, row in df.iterrows()]), total=len(df)))


if __name__ == "__main__":
    main()

100%|██████████| 299150/299150 [1:25:57<00:00, 58.01it/s]


In [13]:
df_ner = df[['pre_process_correct_sentence', 'Masked_sentence']]
df_ner.to_csv(f'pre_processed_mask_data/{file_name}_5l_8l.csv')

### Generate trigrams

In [14]:
def generate_trigrams(text, word_to_combine):
    try:
        words = text.split()
        output = []
        for i in range(len(words)- word_to_combine+1):
            output.append(tuple(words[i:i+word_to_combine]))
        return output
    except TypeError:
        return None

In [15]:
df['trigrams'] = df['Masked_sentence'].progress_apply(lambda x: generate_trigrams(x, 2))

Processing: 100%|██████████| 299150/299150 [00:01<00:00, 269262.43it/s]


In [16]:
trigram_list = df['trigrams'].tolist()
flattened_list = list({item for sublist in trigram_list for item in sublist})

## Saving the JSON file

In [17]:
# # Convert list of tuples to list of lists
# data_list_of_lists = [list(item) for item in flattened_list]

# # Define the file path
# file_path = 'data.json'

# # Write the list of lists to a JSON file
# with open(file_path, 'w') as json_file:
#     json.dump(data_list_of_lists, json_file)

# print(f'Data has been saved to {file_path}')

In [18]:
# # Function to read the JSON file
# def read_json(file_path):
#     try:
#         with open(file_path, 'r') as json_file:
#             data_list_of_lists = json.load(json_file)
#         return [tuple(item) for item in data_list_of_lists]
#     except FileNotFoundError:
#         return []

# # Function to write data to the JSON file
# def write_json(file_path, data):
#     data_list_of_lists = [list(item) for item in data]
#     with open(file_path, 'w') as json_file:
#         json.dump(data_list_of_lists, json_file)

# # Function to append new data to the JSON file
# def append_to_json(file_path, new_data):
#     # Read existing data
#     data = read_json(file_path)
    
#     # Append new data (if it's not already present)
#     for item in tqdm(new_data):
#         if item not in data:
#             data.append(item)
    
#     # Write the updated data back to the JSON file
#     write_json(file_path, data)

In [19]:
# file_path = 'data.json'

In [20]:
# # Append new data to the JSON file
# append_to_json(file_path, flattened_list)

# print(f'New data has been appended to {file_path}')

In [21]:
# def read_json(file_path):
#     try:
#         with open(file_path, 'r') as json_file:
#             data_list_of_lists = json.load(json_file)
#         return [tuple(item) for item in data_list_of_lists]
#     except FileNotFoundError:
#         return []

# def write_json(file_path, data):
#     data_list_of_lists = [list(item) for item in data]
#     with open(file_path, 'w') as json_file:
#         json.dump(data_list_of_lists, json_file)

# def process_chunk(args):
#     chunk, existing_data = args
#     return [item for item in chunk if item not in existing_data]

# def append_to_json(file_path, new_data):
#     # Read existing data
#     existing_data = set(read_json(file_path))
    
#     # Determine the number of CPU cores to use (80% of available cores)
#     num_cores = max(1, int(multiprocessing.cpu_count() * 0.8))
    
#     # Split new_data into chunks
#     chunk_size = max(1, len(new_data) // num_cores)
#     chunks = [new_data[i:i + chunk_size] for i in range(0, len(new_data), chunk_size)]
    
#     # Prepare arguments for process_chunk
#     args = [(chunk, existing_data) for chunk in chunks]
    
#     # Process chunks in parallel
#     with multiprocessing.Pool(num_cores) as pool:
#         results = list(tqdm(pool.imap(process_chunk, args), 
#                             total=len(chunks), desc="Processing chunks"))
    
#     # Combine results
#     new_items = [item for sublist in results for item in sublist]
    
#     # Append new items to existing data
#     updated_data = list(existing_data) + new_items
    
#     # Write the updated data back to the JSON file
#     write_json(file_path, updated_data)

In [22]:
# # Example usage
# if __name__ == '__main__':
#     file_path = 'data1.json'
#     # new_data = [(1, 2, 3), (4, 5, 6), (7, 8, 9)]  # Example new data
#     append_to_json(file_path, flattened_list)

In [23]:
def read_compressed_json(file_path):
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as json_file:
            data_list_of_lists = json.load(json_file)
        return [tuple(item) for item in data_list_of_lists]
    except FileNotFoundError:
        return []

def write_compressed_json(file_path, data):
    data_list_of_lists = [list(item) for item in data]
    with gzip.open(file_path, 'wt', encoding='utf-8') as json_file:
        json.dump(data_list_of_lists, json_file)

def process_chunk(args):
    chunk, existing_data = args
    return [item for item in chunk if item not in existing_data]

def append_to_compressed_json(file_path, new_data):
    # Read existing data
    existing_data = set(read_compressed_json(file_path))
    
    # Determine the number of CPU cores to use (80% of available cores)
    num_cores = max(1, int(multiprocessing.cpu_count() * 0.8))
    
    # Split new_data into chunks
    chunk_size = max(1, len(new_data) // num_cores)
    chunks = [new_data[i:i + chunk_size] for i in range(0, len(new_data), chunk_size)]
    
    # Prepare arguments for process_chunk
    args = [(chunk, existing_data) for chunk in chunks]
    
    # Process chunks in parallel
    with multiprocessing.Pool(num_cores) as pool:
        results = list(tqdm(pool.imap(process_chunk, args), 
                            total=len(chunks), desc="Processing chunks"))
    
    # Combine results
    new_items = [item for sublist in results for item in sublist]
    
    # Append new items to existing data
    updated_data = list(existing_data) + new_items
    
    # Write the updated data back to the compressed JSON file
    write_compressed_json(file_path, updated_data)

In [24]:
# Example usage
if __name__ == '__main__':
    file_path = 'bigrams.gz'  # Note the .gz extension
    # new_data = [(1, 2, 3), (4, 5, 6), (7, 8, 9)]  # Example new data
    append_to_compressed_json(file_path, flattened_list)

Processing chunks: 100%|██████████| 10/10 [00:57<00:00,  5.75s/it]
